In [10]:
import pandas as pd
import enchant   
from pandas import ExcelWriter
from pandas import ExcelFile
from pandas import DataFrame
from openpyxl import Workbook
import xlsxwriter
import math
from collections import Counter
import os

In [12]:
def read_excel(fileName, rowCount, worksheet, ML, successListWorksheet, nameCount_3, failuresListWorksheet, nameCountF_3, nameCount_2, nameCountF_2):
    AL = pd.read_excel(fileName)
    columns_list = AL.columns

    AL.rename(columns = {columns_list[0]:'Attendee', 
                        'Unnamed: 1':'Score','Unnamed: 2':'    1:46 PM','Unnamed: 3':'        1:55 PM','Unnamed: 4':'     2:01 PM',
                        'Unnamed: 5':'       2:41 PM'},inplace=True)
    AL= AL.drop(AL.index [  0:4  ] )
    AL= AL.drop(AL.index [  0:2  ] )
    AL.drop(AL.tail(1).index,inplace=True)
    print(AL)
    master_count=0
    for i in ML.iterrows():
        master_count=master_count+1;
    print(master_count)

    Attendance_count=0
    directmatch=0
#     cosineL=[]
#     cosinematch=0
    n_gram_list=[]
    count_n_gram_match=0
    n_gram_success_3=[]
    n_gram_failure_3=[]
    n_gram_success_count_3=0
    n_gram_failure_count_3=0
    n_gram_success_2=[]
    n_gram_failure_2=[]
    n_gram_success_count_2=0
    n_gram_failure_count_2=0
    
    
    bas_name = os.path.basename(fileName)
    split_name = bas_name.split(".")[0]
    successListWorksheet_3.write(nameCount_3, 0, split_name)
    failuresListWorksheet_3.write(nameCountF_3, 0, split_name)
    nameCount_3 = nameCount_3+1
    nameCountF_3 = nameCountF_3+1
    successListWorksheet_2.write(nameCount_2, 0, split_name)
    failuresListWorksheet_2.write(nameCountF_2, 0, split_name)
    nameCount_2 = nameCount_2+1
    nameCountF_2 = nameCountF_2+1
    countF_3 = 0
    countF_2 = 0
    for i, row in AL.iterrows():
        Attendance_count+=1
        Attendeename=f"{row['Attendee']}"
        upper_case_AN=Attendeename.upper()
        sorted_Attendeename= ''.join(sorted(upper_case_AN))
        ans=900
        flag=0
        for j, row2 in ML.iterrows():
            Mastername=f"{row2['Name']}"
            Scholar_number = f"{row2['Registration Number']}"
            if(Mastername==upper_case_AN):
                flag=1
                break
            
        if(flag==1):
            directmatch=directmatch+1
        else:
            n_gram_list.append(Attendeename)
            count_n_gram_match+=1
    #print(cosineL, attencount, directmatch )
    
    
    def ngram(sentence, num):
        tmp = [] 
        sent_len = len(sentence) - num +1
        for i in range(sent_len):
            tmp.append(sentence[i:i+num]) 
        return tmp


    def diff_ngram(sent_a, sent_b, num):
        a = ngram(sent_a, num)
        b = ngram(sent_b, num) 
        common = [] 
        cnt = 0 
        for i in a:
            for j in b:
                if i == j:
                    cnt += 1
                    common.append(i)
                    break
        return cnt/len(a), common




    for i in n_gram_list:
        attendee_name=i
        ans=-1
        req_list=[]
        upper_case_attendee_name=attendee_name.upper()
        upper_case_removed=upper_case_attendee_name.replace(" ", "")
        sorted_attendee_name= ''.join(sorted(upper_case_removed))
        for j, row2 in ML.iterrows():
            Master_name=f"{row2['Name']}"
            Scholar_number = f"{row2['Registration Number']}"
            master_name_removed=Master_name.replace(" ", "")
            sorted_master_name= ''.join(sorted(master_name_removed))
            val,val_list=diff_ngram(str(sorted_attendee_name), str(sorted_master_name),3)
            if(val>=ans):
                ans=val
                resultant_mastername= Master_name
                resultant_scholarnumber= Scholar_number
        print(attendee_name+"   "+resultant_mastername+"  "+str(ans))
        
        if(ans==1.0):
            n_gram_success_count_3+=1
            successListWorksheet_3.write(nameCount_3, 0, resultant_mastername)
            successListWorksheet_3.write(nameCount_3, 1, resultant_scholarnumber)
            nameCount_3 = nameCount_3+1  
        else:
            n_gram_failure_count_3+=1
            failuresListWorksheet_3.write(nameCountF_3, 0, attendee_name)
            countF_3 = countF_3+1
            nameCountF_3 = nameCountF_3+1
            
            
    for i in n_gram_list:
        attendee_name=i
        ans=-1
        req_list=[]
        upper_case_attendee_name=attendee_name.upper()
        upper_case_removed=upper_case_attendee_name.replace(" ", "")
        sorted_attendee_name= ''.join(sorted(upper_case_removed))
        for j, row2 in ML.iterrows():
            Master_name=f"{row2['Name']}"
            Scholar_number = f"{row2['Registration Number']}"
            master_name_removed=Master_name.replace(" ", "")
            sorted_master_name= ''.join(sorted(master_name_removed))
            val,val_list=diff_ngram(str(sorted_attendee_name), str(sorted_master_name),2)
            if(val>=ans):
                ans=val
                resultant_mastername= Master_name
                resultant_scholarnumber= Scholar_number
        print(attendee_name+"   "+resultant_mastername+"  "+str(ans))
        
        if(ans==1.0):
            n_gram_success_count_2+=1
            successListWorksheet_2.write(nameCount_2, 0, resultant_mastername)
            successListWorksheet_2.write(nameCount_2, 1, resultant_scholarnumber)
            nameCount_2 = nameCount_2+1
        else:
            n_gram_failure_count_2+=1
            failuresListWorksheet_2.write(nameCountF_2, 0, attendee_name)
            countF_2 = countF_2+1
            nameCountF_2 = nameCountF_2+1
            
    if countF_3 == 0:
        failuresListWorksheet_3.write(nameCountF_3, 0, "No Failures")
        nameCountF_3 = nameCountF_3+1
        
    if countF_2 == 0:
        failuresListWorksheet_2.write(nameCountF_2, 0, "No Failures")
        nameCountF_2 = nameCountF_2+1    
        
    #print(cosineS1,cosineS2,cosineF)
    nameCount_3 = nameCount_3+1 
    nameCountF_3 = nameCountF_3+1
    nameCount_2 = nameCount_2+1 
    nameCountF_2 = nameCountF_2+1
            
     
    master_count
    x=Attendance_count/master_count
    y=x*100
    total_per=round(y,2)
    directmatch
    total_per
    per_dir_str_match=directmatch/Attendance_count 
    per_dir_str_match=per_dir_str_match*100
    per_dir_str_match=round(per_dir_str_match,2)
    n_gram_str_match=(count_n_gram_match)/Attendance_count
    n_gram_str_match=n_gram_str_match*100
    n_gram_str_match=round(n_gram_str_match,2)
    n_gram_success_percentage_3=n_gram_success_count_3/Attendance_count
    n_gram_success_percentage_3=n_gram_success_percentage_3*100
    n_gram_success_percentage_3=round(n_gram_success_percentage_3,2)
    n_gram_success_percentage_3
    n_gram_failure_percentage_3=n_gram_failure_count_3/Attendance_count
    n_gram_failure_percentage_3=n_gram_failure_percentage_3*100
    n_gram_failure_percentage_3=round(n_gram_failure_percentage_3,2)
    n_gram_failure_percentage_3
    n_gram_success_percentage_2=n_gram_success_count_2/Attendance_count
    n_gram_success_percentage_2=n_gram_success_percentage_2*100
    n_gram_success_percentage_2=round(n_gram_success_percentage_2,2)
    n_gram_success_percentage_2
    n_gram_failure_percentage_2=n_gram_failure_count_2/Attendance_count
    n_gram_failure_percentage_2=n_gram_failure_percentage_2*100
    n_gram_failure_percentage_2=round(n_gram_failure_percentage_2,2)
    n_gram_failure_percentage_2
    
    worksheet.write(0, 0,'Date')
    worksheet.write(0, 1, 'Total percentage of attendee')
    worksheet.write(0, 2, 'Total percentage of direct string match' )
    worksheet.write(0, 3, 'Total percentage of n-gram match' )
    worksheet.write(0, 4, 'percentage of 3-gram match with value as 1.0' )
    worksheet.write(0, 5, 'Failure percentage with 3-gram(Value < 1.0)' )
    worksheet.write(0, 6, 'percentage of 2-gram match with value as 1.0' )
    worksheet.write(0, 7, 'Failure percentage with 2-gram(Value < 1.0)' )
    worksheet.write(0, 8, 'Number of Attendee')
    worksheet.write(0, 9, 'Number of direct string match')
    worksheet.write(0, 10, 'Number of n-gram string match')
    worksheet.write(0, 11, 'Number of success with 3-gram (Value==1.0)')
    worksheet.write(0, 12, 'Number of failure with 3-gram( Value<1.0)')
    worksheet.write(0, 13, 'Number of success with 2-gram (Value==1.0)')
    worksheet.write(0, 14, 'Number of failure with 2-gram( Value<1.0)')
    
    date_file = fileName.split(".")
    date_tbu = date_file[0]

    worksheet.write(rowCount,0,date_tbu)
    worksheet.write(rowCount, 1, total_per)
    worksheet.write(rowCount, 2, per_dir_str_match )
    worksheet.write(rowCount, 3, n_gram_str_match)
    worksheet.write(rowCount, 4, n_gram_success_percentage_3)
    worksheet.write(rowCount, 5, n_gram_failure_percentage_3)
    worksheet.write(rowCount, 6, n_gram_success_percentage_2)
    worksheet.write(rowCount, 7, n_gram_failure_percentage_2)
    worksheet.write(rowCount, 8, Attendance_count )
    worksheet.write(rowCount, 9, directmatch)
    worksheet.write(rowCount, 10, count_n_gram_match)
    worksheet.write(rowCount, 11, n_gram_success_count_3)
    worksheet.write(rowCount, 12, n_gram_failure_count_3)
    worksheet.write(rowCount, 13, n_gram_success_count_2)
    worksheet.write(rowCount, 14, n_gram_failure_count_2)
    return (nameCount_3, nameCountF_3,nameCount_2, nameCountF_2)

def checkIfXlsx(fileName):
    fileData = fileName.split(".")
    fileExt = fileData[-1]
    fileBaseName = os.path.basename(fileName)
    if(fileBaseName=='n-gram_2_Set_temp.xlsx' or fileBaseName=='MasterStudentList-2020.xlsx' or
        fileBaseName=='Output_Success_3gram.xlsx_temp' or fileBaseName=='Output_Failures_3gram_temp.xlsx'  or
        fileBaseName=='Output_Success_2gram.xlsx_temp' or fileBaseName=='Output_Failures_2gram_temp.xlsx'):
        return False
    if(fileExt=='xlsx'):
        return True
    return False

if __name__ == '__main__':
    ML = pd.read_excel("MasterStudentList-2020.xlsx")
    workbook = xlsxwriter.Workbook("n-gram_2_Set_temp.xlsx",)
    successListWorkbook_3 = xlsxwriter.Workbook("Output_Success_3gram_temp.xlsx")
    failuresListWorkbook_3 = xlsxwriter.Workbook("Output_Failures_3gram_temp.xlsx")
    successListWorkbook_2 = xlsxwriter.Workbook("Output_Success_2gram_temp.xlsx")
    failuresListWorkbook_2 = xlsxwriter.Workbook("Output_Failures_2gram_temp.xlsx")
    worksheet = workbook.add_worksheet()
    successListWorksheet_3 = successListWorkbook_3.add_worksheet()
    failuresListWorksheet_3 = failuresListWorkbook_3.add_worksheet()
    successListWorksheet_2 = successListWorkbook_2.add_worksheet()
    failuresListWorksheet_2 = failuresListWorkbook_2.add_worksheet()
    dir_list = os.listdir()
    fileCount = 0
    nameCount_3 = 0
    nameCountF_3 = 0
    nameCount_2 = 0
    nameCountF_2 = 0
    for file in dir_list:
        if(checkIfXlsx(fileName=file)):
            fileCount = fileCount+1
            (camCount_3, camCount_f_3, camCount_2, camCount_f_2 ) = read_excel(file, fileCount, worksheet, ML, successListWorksheet, nameCount_3, failuresListWorksheet, nameCountF_3, nameCount_2 ,nameCountF_2)
            nameCount_3 = camCount_3
            nameCountF_3 = camCount_f_3
            nameCount_2 = camCount_2
            nameCountF_2 = camCount_f_2
            
            
    workbook.close()
    successListWorkbook_3.close()
    failuresListWorkbook_3.close()
    successListWorkbook_2.close()
    failuresListWorkbook_2.close()

                     Attendee Score     1:46 PM         1:55 PM
6             ADAPAKALA AKASH   2/2           ✅               ✅
7                 ASISH PADHI   2/2           ✅               ✅
8               BODALA RAJESH   2/2           ✅               ✅
9   BYRRAJU SATISH KUMAR RAJU   2/2           ✅               ✅
10             Chandan Mishra   2/2           ✅               ✅
11          DUKKA TEJEPRAKASH   2/2           ✅               ✅
12         Gedela Devi Prasad   2/2           ✅               ✅
13          GOUTAM KUMAR DASH   2/2           ✅               ✅
14            HARSHITA MISHRA   2/2           ✅               ✅
15        INDUKURI ANIL VARMA   2/2           ✅               ✅
16                JAMI MUKESH   2/2           ✅               ✅
17    KALINGO AUROBINDO NAYAK   2/2           ✅               ✅
18          KELLA KIRAN KUMAR   2/2           ✅               ✅
19         KENGAM CHIRANJEEVI   2/2           ✅               ✅
20          KODI RAMA KRISHNA   2/2     

DIPTI JENA   DIPTI  JENA  1.0
INDUKURI ANIL VARMA   INDUKURI ANIL  VARMA  1.0
KALINGO AUROBINDO NAYAK   KALINGO AUROBINDO  NAYAK  1.0
KILLARI YOGESH   KILLARI  YOGESH  1.0
KODI RAMA KRISHNA   KODI RAMA  KRISHNA  1.0
MEGHANA PATNANA   MEGHANA  PATNANA  1.0
Penabakala Neeraja   PENABAKALA  NEERAJA  1.0
S SAI KIRAN   S SAI  KIRAN  1.0
Shankar Pradhan   SHANKAR  PRADHAN  1.0
SILLA.PADMAJA   SILLA.PADMAJA    1.0
SRIJA TRIPATHY   SRIJA  TRIPATHY  1.0
SUVENDU KUMAR SAHOO   SUVENDU KUMAR  SAHOO  1.0
Tangudu Sravani   TANGUDU  SRAVANI  1.0
CHINMAYEE MISHRA   CHINMAYEE  MISHRA  1.0
Raj chandan Behera   TANASHREE  SUARA  0.2857142857142857
ABDUL NASEER   ABDUL  NASEER  1.0
ASISH PADHI   ASISH  PADHI  1.0
BODALA RAJESH   BODALA  RAJESH  1.0
BYRRAJU SATISH KUMAR RAJU   BYRRAJU SATISH KUMAR  RAJU  1.0
DIPTI JENA   DIPTI  JENA  1.0
INDUKURI ANIL VARMA   INDUKURI ANIL  VARMA  1.0
KALINGO AUROBINDO NAYAK   KALINGO AUROBINDO  NAYAK  1.0
KILLARI YOGESH   KILLARI  YOGESH  1.0
KODI RAMA KRISHNA   KODI RAMA

SHIREESHA IDDUMU   SHIREESHA  IDDUMU  1.0
DIPTI JENA   DIPTI  JENA  1.0
Gedela Devi Prasad   GEDELA  DEVI PRASAD  1.0
MENDA PAVAN KUMAR   MENDA PAVAN  KUMAR  1.0
Shankar Pradhan   SHANKAR  PRADHAN  1.0
SUVENDU KUMAR SAHOO   SUVENDU KUMAR  SAHOO  1.0
TRIPTI KUMARI   TRIPTI  KUMARI  1.0
                     Attendee Score     1:46 PM         1:55 PM      2:01 PM
6                ABDUL NASEER   3/3           ✅               ✅            ✅
7             ADAPAKALA AKASH   3/3           ✅               ✅            ✅
8              Chandan Mishra   3/3           ✅               ✅            ✅
9                  DIPTI JENA   3/3           ✅               ✅            ✅
10         Gedela Devi Prasad   2/3           ✅               ✅            ❎
11            HARSHITA MISHRA   3/3           ✅               ✅            ✅
12        INDUKURI ANIL VARMA   3/3           ✅               ✅            ✅
13                JAMI MUKESH   3/3           ✅               ✅            ✅
14          KELLA KIR

ABDUL NASEER   ABDUL  NASEER  1.0
ADAPAKALA AKASH   ADAPAKALA  AKASH  1.0
ASISH PADHI   ASISH  PADHI  1.0
BYRRAJU SATISH KUMAR RAJU   BYRRAJU SATISH KUMAR  RAJU  1.0
Chandan Mishra   CHANDAN  MISHRA  1.0
CHINMAYEE MISHRA   CHINMAYEE  MISHRA  1.0
DIPTI JENA   DIPTI  JENA  1.0
HARSHITA MISHRA   HARSHITA  MISHRA  1.0
INDUKURI ANIL VARMA   INDUKURI ANIL  VARMA  1.0
JAMI MUKESH   JAMI  MUKESH  1.0
KALINGO AUROBINDO NAYAK   KALINGO AUROBINDO  NAYAK  1.0
KELLA KIRAN KUMAR   KELLA  KIRAN KUMAR  1.0
KENGAM CHIRANJEEVI   KENGAM  CHIRANJEEVI  1.0
KODI RAMA KRISHNA   KODI RAMA  KRISHNA  1.0
LIPIKA PRADHAN   LIPIKA  PRADHAN  1.0
MANDAVALLI SUMANTH   MANDAVALLI  SUMANTH  1.0
MENDA PAVAN KUMAR   MENDA PAVAN  KUMAR  1.0
NILAKANTHA PRADHAN   NILAKANTHA  PRADHAN  1.0
Nitin Kumar   NITIN  KUMAR  1.0
Om Prakash Kumar   OM  PRAKASH KUMAR  1.0
PAVAN BARATAM   BARATRAM  PAVAN  0.9
PITABASH PRADHAN   PITABASH  PRADHAN  1.0
S SAI KIRAN   S SAI  KIRAN  1.0
Shankar Pradhan   SHANKAR  PRADHAN  1.0
SILLA.PADMAJA  

OM PRAKASH KHILLO   OM  PRAKASH KHILLO  1.0
SUVENDU KUMAR SAHOO   SUVENDU KUMAR  SAHOO  1.0
KODI RAMA KRISHNA   KODI RAMA  KRISHNA  1.0
ABDUL NASEER   ABDUL  NASEER  1.0
ADAPAKALA AKASH   ADAPAKALA  AKASH  1.0
BYRRAJU SATISH KUMAR RAJU   BYRRAJU SATISH KUMAR  RAJU  1.0
Chandan Mishra   CHANDAN  MISHRA  1.0
CHINMAYEE MISHRA   CHINMAYEE  MISHRA  1.0
DIPTI JENA   DIPTI  JENA  1.0
DUKKA TEJEPRAKASH   DUKKA  TEJAPRAKASH  0.9333333333333333
Gedela Devi Prasad   GEDELA  DEVI PRASAD  1.0
HARSHITA MISHRA   HARSHITA  MISHRA  1.0
INDUKURI ANIL VARMA   INDUKURI ANIL  VARMA  1.0
JAMI MUKESH   JAMI  MUKESH  1.0
KALINGO AUROBINDO NAYAK   KALINGO AUROBINDO  NAYAK  1.0
KELLA KIRAN KUMAR   KELLA  KIRAN KUMAR  1.0
KENGAM CHIRANJEEVI   KENGAM  CHIRANJEEVI  1.0
KILLARI YOGESH   KILLARI  YOGESH  1.0
MANDAVALLI SUMANTH   MANDAVALLI  SUMANTH  1.0
MEGHANA PATNANA   MEGHANA  PATNANA  1.0
NILAKANTHA PRADHAN   NILAKANTHA  PRADHAN  1.0
Nitin Kumar   NITIN  KUMAR  1.0
Om Prakash Kumar   OM  PRAKASH KUMAR  1.0
PABIT

...

TRIPTI KUMARI   TRIPTI  KUMARI  1.0
ASISH PADHI   ASISH  PADHI  1.0
DIPTI JENA   DIPTI  JENA  1.0
DUKKA TEJEPRAKASH   DUKKA  TEJAPRAKASH  0.8571428571428571
JAMI MUKESH   JAMI  MUKESH  1.0
KENGAM CHIRANJEEVI   KENGAM  CHIRANJEEVI  1.0
Tukuna Pradhan   TUKUNA  PRADHAN  1.0
DOLA CHARAN   DOLA  CHARAN  1.0
ABDUL NASEER   ABDUL  NASEER  1.0
ADAPAKALA AKASH   ADAPAKALA  AKASH  1.0
Chandan Mishra   CHANDAN  MISHRA  1.0
HARSHITA MISHRA   HARSHITA  MISHRA  1.0
INDUKURI ANIL VARMA   INDUKURI ANIL  VARMA  1.0
KANAKALA SIVAMANI   KANAKALA  SIVAMANI  1.0
KODI RAMA KRISHNA   KODI RAMA  KRISHNA  1.0
LIPIKA PRADHAN   LIPIKA  PRADHAN  1.0
MANDAVALLI SUMANTH   MANDAVALLI  SUMANTH  1.0
MEGHANA PATNANA   MEGHANA  PATNANA  1.0
NILAKANTHA PRADHAN   NILAKANTHA  PRADHAN  1.0
Nitin Kumar   NITIN  KUMAR  1.0
Om Prakash Kumar   OM  PRAKASH KUMAR  1.0
PAVAN BARATAM   BARATRAM  PAVAN  1.0
PITABASH PRADHAN   PITABASH  PRADHAN  1.0
S SAI KIRAN   S SAI  KIRAN  1.0
SHAKTI PRASAD NAHAK   SHAKTI  PRASAD NAHAK  1.0
Shek

SRI HARSHA ANDHAVARAPU   ANDHAVARAPU  SRI HARSHA  1.0
SRIJA TRIPATHY   SRIJA  TRIPATHY  1.0
SUNIL KUMAR   SUNIL  KUMAR  1.0
SUSHANT SHEKHAR   SUSHANT  SHEKHAR  1.0
TALABHAKTULA SAIKIRAN   TALABHAKTULA SAI KIRAN  1.0
Tangudu Sravani   TANGUDU  SRAVANI  1.0
TANUSHREE SUARA   TANASHREE  SUARA  0.9230769230769231
TRIPTI KUMARI   TRIPTI  KUMARI  1.0
Tukuna Pradhan   TUKUNA  PRADHAN  1.0
VIKASH KUMAR   VIKASH  KUMAR  1.0
VIMLESH KUMAR   VIMLESH  KUMAR  1.0
VINEET KUMAR   VINEET  KUMAR  1.0
YEDDLA SURESH KUMAR   YEDDLA SURESH  KUMAR  1.0
                     Attendee Score     1:46 PM         1:55 PM      2:01 PM
6                ABDUL NASEER   3/3           ✅               ✅            ✅
7             ADAPAKALA AKASH   3/3           ✅               ✅            ✅
8                 ASISH PADHI   3/3           ✅               ✅            ✅
9               BODALA RAJESH   3/3           ✅               ✅            ✅
10             Chandan Mishra   3/3           ✅               ✅            ✅
1

ABDUL NASEER   ABDUL  NASEER  1.0
ADAPAKALA AKASH   ADAPAKALA  AKASH  1.0
ADITI JHA   ADITI  JHA  1.0
AMISHA SOREN   AMISHA  SOREN  1.0
ARPITA MALLICK   ARPITA  MALLICK  1.0
ASISH PADHI   ASISH  PADHI  1.0
BHARTI KUMARI   BHARTI  KUMARI  1.0
BODALA RAJESH   BODALA  RAJESH  1.0
BYRRAJU SATISH KUMAR RAJU   BYRRAJU SATISH KUMAR  RAJU  1.0
CHINMAYEE MISHRA   CHINMAYEE  MISHRA  1.0
DHARMAVARAPU SHIVA   DHARMAVARAPU  SHIVA  1.0
DIPTI JENA   DIPTI  JENA  1.0
DOLA CHARAN   DOLA  CHARAN  1.0
DUKKA TEJEPRAKASH   DUKKA  TEJAPRAKASH  0.8571428571428571
Gedela Devi Prasad   GEDELA  DEVI PRASAD  1.0
GOPAL KUMAR   GOPAL  KUMAR  1.0
GURUPRIT KUMAR   GURUPRIT  KUMAR  1.0
HARSHITA MISHRA   HARSHITA  MISHRA  1.0
INDUKURI ANIL VARMA   INDUKURI ANIL  VARMA  1.0
JAMI BHUJANGA RAO   JAMI BHUJANGA  RAO  1.0
JAMI MUKESH   JAMI  MUKESH  1.0
K. UMA SHANKAR   ANKUSH  KUMAR  0.8
KALINGO AUROBINDO NAYAK   KALINGO AUROBINDO  NAYAK  1.0
KANAKALA SIVAMANI   KANAKALA  SIVAMANI  1.0
KELLA KIRAN KUMAR   KELLA  KIRAN KUMA

RITESH KUMAR GOSAIN   RITESH KUMAR GOSAI  0.8
S MANOJ   JAMI BHUJANGA  RAO  0.5
SHIREESHA IDDUMU   SHIREESHA  IDDUMU  1.0
SILLA.PADMAJA   SILLA.PADMAJA    1.0
SIREESHA AGURU   AGURU  SIREESHA  1.0
SONTANA GEETASRI   SONTANA  GEETASRI  1.0
SRI HARSHA ANDHAVARAPU   ANDHAVARAPU  SRI HARSHA  1.0
SRIJA TRIPATHY   SRIJA  TRIPATHY  1.0
SUNIL KUMAR   SUNIL  KUMAR  1.0
Tangudu Sravani   TANGUDU  SRAVANI  1.0
VINEET KUMAR   VINEET  KUMAR  1.0
YEDDLA SURESH KUMAR   YEDDLA SURESH  KUMAR  1.0
ADITI JHA   ADITI  JHA  1.0
AMISHA SOREN   AMISHA  SOREN  1.0
ASISH PADHI   ASISH  PADHI  1.0
BASANT KUMAR   BASANT  KUMAR  1.0
BHARTI KUMARI   BHARTI  KUMARI  1.0
BODALA RAJESH   BODALA  RAJESH  1.0
Chandan Mishra   CHANDAN  MISHRA  1.0
Deepak Kumar Patra   DEEPAK KUMAR PATRO  0.8571428571428571
DUKKA TEJEPRAKASH   DUKKA  TEJAPRAKASH  0.8571428571428571
GARUGU MANIKANTA   GARUGU  MANIKANTA  1.0
GURUPRIT KUMAR   GURUPRIT  KUMAR  1.0
HARSHITA MISHRA   HARSHITA  MISHRA  1.0
JAMI BHUJANGA RAO   JAMI BHUJANGA  RAO

ASISH PADHI   ASISH  PADHI  1.0
BASANT KUMAR   BASANT  KUMAR  1.0
BHARTI KUMARI   BHARTI  KUMARI  1.0
BODALA RAJESH   BODALA  RAJESH  1.0
BYRRAJU SATISH KUMAR RAJU   BYRRAJU SATISH KUMAR  RAJU  1.0
Chandan Mishra   CHANDAN  MISHRA  1.0
DHARMAVARAPU SHIVA   DHARMAVARAPU  SHIVA  1.0
DOLA CHARAN   DOLA  CHARAN  1.0
DOLA DIWAKAR   DOLA  DIWAKAR  1.0
DUKKA TEJEPRAKASH   DUKKA  TEJAPRAKASH  0.9333333333333333
GOPAL KUMAR   GOPAL  KUMAR  1.0
GURUPRIT KUMAR   GURUPRIT  KUMAR  1.0
HARSHITA MISHRA   HARSHITA  MISHRA  1.0
INDUKURI ANIL VARMA   INDUKURI ANIL  VARMA  1.0
JAMI BHUJANGA RAO   JAMI BHUJANGA  RAO  1.0
JAMI MUKESH   JAMI  MUKESH  1.0
JATIN BISOI   JATIN  BISOI  1.0
K. UMA SHANKAR   ANKUSH  KUMAR  0.9090909090909091
KALINGO AUROBINDO NAYAK   KALINGO AUROBINDO  NAYAK  1.0
KANAKALA SIVAMANI   KANAKALA  SIVAMANI  1.0
KELLA KIRAN KUMAR   KELLA  KIRAN KUMAR  1.0
KENGAM CHIRANJEEVI   KENGAM  CHIRANJEEVI  1.0
KHUSHBOO KUMARI   KHUSHBOO  KUMARI  1.0
KILLARI YOGESH   KILLARI  YOGESH  1.0
KODI RAM